### 状変データ解析

In [13]:
import datetime
import csv

#### 文字列の整形 (佐藤さんのコード)

In [24]:
# from Sato-san's ana-APPARATUS_STATUS.py
# define Status ID to Status NAME
def sid2sname(id):
    try:
        ret = snamedict[id]
    except:
        ret = "Unknown Status"
    return ret

# define Device ID to Device NAME
def did2dname(id):
    try:
        ret = dnamedict[id]
    except:
        ret = "Undefined Device"
    return ret

# define isALARM
def isAlarm(id):
    try:
        ret = alarmdict[id]
    except:
        ret = "NORMAL"
    return ret

# define Status ID to Status Type
def sid2stype(id):
    try:
        ret = stypedict[id]
    except:
        ret = "??"
    return ret
    
# Read Status List and append dictionary
with open('STATUS.TABLE.tsv', 'r') as f:
    f1dat = csv.reader(f, delimiter = '\t')
    snamedict = {}
    for row in f1dat:
    	snamedict[row[0]] = row[1]

# Read Status List and append dictionary
with open('ID-LOCAL.tsv', 'r') as f:
    f2dat = csv.reader(f, delimiter = '\t')
    dnamedict = {}
    for row in f2dat:
    	dnamedict[row[0]] = row[1]

# Read Alarm Status ID and append dictionary
with open('ALARM-CAUTION-FAULT.tsv', 'r') as f:
    f3dat = csv.reader(f, delimiter = '\t')
    alarmdict = {}
    for row in f3dat:
    	alarmdict[row[0]] = row[1]
# Read Alarm Status ID and append dictionary
with open('SID-STYPE.tsv', 'r') as f:
    f4dat = csv.reader(f, delimiter = '\t')
    stypedict = {}
    for row in f4dat:
    	stypedict[row[0]] = row[1]


In [82]:
# from Sato-san's ana-APPARATUS_STATUS.py
def formatMessage(flg, sid, sname, strHst):
    d = ' '
    did = sid[:4]	# Device ID
    dname = did2dname(did)
    alm = isAlarm(sid)
    stype = sid2stype(sid)
    result = f'{strHst}{d}{stype}{d}{alm:<6}{d}0x{sid}{d}{dname:<4}{d}{sname:<32}{d}{flg}'
    return result


#### 調査対象の時刻レンジと検索キーワードの設定

In [94]:
startDateTime = datetime.datetime(2024, 3, 20, 4, 30, 0)
durationInSec = 3600
keyWords = ["ALARM", "MCU"]
#keyWords = ["ALARM", "Hydro"]

stopDateTime = startDateTime + datetime.timedelta(seconds=durationInSec)
# a log file covers the time range of HST 14:00: 14:00+1d

if startDateTime.strftime('%p') == 'PM':
    logDay = startDateTime  + datetime.timedelta(days=1)
else:
    # AM
    logDay = startDateTime

# /tscbin/tsc/2024/APPARATUS_STATUS.20240518.LOG
filePath = f'/tscbin/tsc/{logDay.year}/APPARATUS_STATUS.{logDay.year}{logDay.month:02d}{logDay.day:02d}.LOG'
print(filePath)
with open(filePath) as f:
    lines = [s.rstrip() for s in f.readlines()] # chop the last CR


for line in lines:
    t = line[0:15]    # UT
    flg = line[32:35] # OCR or RCV
    sid = line[38:45] # Status ID
    sname = line[46:] 

    date = datetime.date(int(t[:4]), int(t[4:6]), int(t[6:8]))
    time = datetime.time(int(t[8:10]), int(t[10:12]), int(t[12:14]), int(t[14:])*100000)
    ut  = datetime.datetime.combine(date, time)
    hst = ut - datetime.timedelta(hours=10)

    if (hst < startDateTime) or (hst > stopDateTime):
        continue
        
    strHst = hst.strftime('%Y-%m-%d %H:%M:%S.%f')[:21]

    result = formatMessage(flg, sid, sname, strHst)

    numberOfMatch = 0
    for k in keyWords:
        if (k in result) == True:
            numberOfMatch += 1
    if numberOfMatch == len(keyWords):
        print(result)

/tscbin/tsc/2024/APPARATUS_STATUS.20240320.LOG
2024-03-20 05:04:10.8 E ALARM  0x0002022 MCU  Mount Pointing Mode              OCR
2024-03-20 05:04:10.8 E ALARM  0x000210D MCU  Mount Fault                      OCR
2024-03-20 05:04:10.8 E ALARM  0x000213A MCU  Mount S/D REF Alarm              OCR
2024-03-20 05:04:10.8 E ALARM  0x000213B MCU  AZ CRS Alarm                     OCR
2024-03-20 05:04:10.8 E ALARM  0x000213C MCU  AZ FINE Alarm                    OCR
2024-03-20 05:04:10.8 E ALARM  0x000213D MCU  EL CRS Alarm                     OCR
2024-03-20 05:04:10.8 E ALARM  0x000213E MCU  EL FINE Alarm                    OCR


In [1]:
import datetime
from telDav import ApparatusStatus

apparatusStatus = ApparatusStatus.ApparatusStatus(startDatetime=datetime.datetime(2024, 3, 20, 4, 50, 0),
                                                  stopDatetime=datetime.datetime(2024, 3, 20, 5, 30, 0),
                                                  keyWords=["ALARM", "MCU"])
lines = apparatusStatus.run()
for l in lines:
    print(l)

/tscbin/tsc/2024/APPARATUS_STATUS.20240320.LOG
2024-03-20 05:04:10.8 E ALARM  0x0002022 MCU  Mount Pointing Mode              OCR
2024-03-20 05:04:10.8 E ALARM  0x000210D MCU  Mount Fault                      OCR
2024-03-20 05:04:10.8 E ALARM  0x000213A MCU  Mount S/D REF Alarm              OCR
2024-03-20 05:04:10.8 E ALARM  0x000213B MCU  AZ CRS Alarm                     OCR
2024-03-20 05:04:10.8 E ALARM  0x000213C MCU  AZ FINE Alarm                    OCR
2024-03-20 05:04:10.8 E ALARM  0x000213D MCU  EL CRS Alarm                     OCR
2024-03-20 05:04:10.8 E ALARM  0x000213E MCU  EL FINE Alarm                    OCR
